# Export Files for GNPS

With pyOpenMS you can automatically generate all files needed for GNPS
Feature-Based Molecular Networking (FBMN) and Ion Identity Molecular
Networking (IIMN).

Pre-requisites are your input
[mzML](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-mzML)
files and a
[ConsensusMap](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.ConsensusMap.html),
generated by an [untargeted metabolomics pre-processing
workflow](untargeted_metabolomics_preprocessing.ipynb). Ensure that
[MS2](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-MS2)
data has been mapped to the
[FeatureMap](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.FeatureMap.html)
objects with
[IDMapper](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.IDMapper.html).
For IIMN adduct detection must have been performed on the
[FeatureMap](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.FeatureMap.html)
objects during pre-processing with
[MetaboliteFeatureDeconvolution](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.MetaboliteFeatureDeconvolution.html).

First, download two example
[mzML](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-mzML)
files that have been map aligned based on a
[feature map](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-feature-map)
alignment.

In [ ]:
from urllib.request import urlretrieve

gh = "https://raw.githubusercontent.com/OpenMS/pyopenms-docs/master"
urlretrieve(
    gh + "/src/data/Metabolomics_1_aligned.mzML", "Metabolomics_1_aligned.mzML"
)
urlretrieve(
    gh + "/src/data/Metabolomics_2_aligned.mzML", "Metabolomics_2_aligned.mzML"
)
urlretrieve(
    gh + "/src/data/UntargetedMetabolomics.consensusXML",
    "UntargetedMetabolomics.consensusXML",
)

In [ ]:
import pyopenms as oms

mzML_files = ["Metabolomics_1_aligned.mzML", "Metabolomics_2_aligned.mzML"]

consensusXML_file = "UntargetedMetabolomics.consensusXML"

Since GNPS only works with features that contain
[MS2](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-MS2)
fragmentation spectra, the first step is to filter out features from
your
[ConsensusMap](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.ConsensusMap.html)
that have no
[MS2](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-MS2)
spectra annotated.

In [ ]:
consensus_map = oms.ConsensusMap()
oms.ConsensusXMLFile().load(consensusXML_file, consensus_map)
filtered_map = oms.ConsensusMap(consensus_map)
filtered_map.clear(False)
for feature in consensus_map:
    if feature.getPeptideIdentifications():
        filtered_map.push_back(feature)

consensusXML_file = "filtered.consensusXML"
oms.ConsensusXMLFile().store(consensusXML_file, filtered_map)

Now you can export your all files for FBMN and IIMN.

In [ ]:
# for FFBM
oms.GNPSMGFFile().store(
    oms.String(consensusXML_file),
    [file.encode() for file in mzML_files],
    oms.String("MS2data.mgf"),
)
oms.GNPSQuantificationFile().store(consensus_map, "FeatureQuantificationTable.txt")
oms.GNPSMetaValueFile().store(consensus_map, "MetaValueTable.tsv")

# for IIMN
oms.IonIdentityMolecularNetworking().annotateConsensusMap(consensus_map)
oms.IonIdentityMolecularNetworking().writeSupplementaryPairTable(
    consensus_map, "SupplementaryPairTable.csv"
)